# Example Notebook to showcasing how we interact with JTReaders

In [1]:
# First change dir to JTR parent
import os
os.chdir('..')

### Bookkeeping of all existing readers: `readers.py`

In [2]:
import jack.readers as readers

In [3]:
print("Existing models:\n%s" % ", ".join(readers.readers.keys()))

Existing models:
esim_snli_reader, distmult_reader, cbow_xqa_reader, cbilstm_snli_reader, fastqa_reader, modelf_reader, dam_snli_reader, transe_reader, complex_reader, bidaf_reader


### Create a reader

In [4]:
%%script bash
bash data/GloVe/download_small.sh

glove.6B.50d.txt already exists! Doing nothing!


In [5]:
from jack.io.embeddings.embeddings import load_embeddings
from jack.util.vocab import Vocab

# we need a vocabulary (with embeddings for our fastqa_reader, but this is not always necessary)
embeddings = load_embeddings('data/GloVe/glove.6B.50d.txt', 'glove')
vocab = Vocab(emb=embeddings, init_from_embeddings=True)

# and a config
config = {"repr_dim": 10, "repr_dim_input": embeddings.lookup.shape[1], "model": "fastqa_reader"}

In [6]:
# create example reader
from jack.core import SharedResources

svac = SharedResources(vocab, config)
fastqa_reader = readers.readers["fastqa_reader"](svac)

### Setting up a reader from training data 

In [7]:
from jack.io.load import load_jack
train_data = load_jack('data/SQuAD/snippet.jtr.json')
# all parameters are initialized after this call
fastqa_reader.setup_from_data(train_data)

### Saving the reader

In [8]:
fastqa_reader.store("/tmp/fastqa_reader")

In [9]:
%%sh
ls /tmp/fastqa_reader/

checkpoint
model_module.data-00000-of-00001
model_module.index
model_module.meta
shared_resources
shared_resources_vocab


### Loading the reader

In [10]:
# we can simply load a setup reader
fastqa_reader.load("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


In [11]:
import tensorflow as tf
# reset graph -> computation graph is gone
tf.reset_default_graph()

# or setup a new reader from file 
svac = SharedResources()
fastqa_reader = readers.readers["fastqa_reader"](svac)
fastqa_reader.load_and_setup("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


In [12]:
# reset graph -> computation graph is gone
tf.reset_default_graph()

# or even shorter, use the utility function for creating and loading a reader from file
fastqa_reader = readers.reader_from_file("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


### Applying the reader

In [13]:
from jack.io.load import load_jack
data = load_jack('data/SQuAD/snippet.jtr.json')

# take a list of inputs, e.g., from our training data
questions = [q for q, a in data]

In [14]:
for a in fastqa_reader(questions):
    print("{}, {}, {}".format(a.score, a.span, a.text))

0.00011519215331645682, (242, 492), legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary
0.00011816684127552435, (242, 319), legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the
0.00012097939179511741, (242, 326), legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred
0.00011362232180545107, (242, 487), legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin
0.00011911606998182833, (242, 492), legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of pra